In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git
!wget https://pjreddie.com/media/files/yolov5.weights

In [ ]:
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# Load YOLOv3  
def YoloLoad():
    dnet = cv2.dnn.readNet("darknet/yolov5.weights", "darknet/cfg/yolov5.cfg")
    with open("darknet/data/coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    return dnet, classes

In [ ]:
# Preprocess image for YOLO
def imgPreprocess(imgPath):
    img = cv2.imread(imgPath)
    height, width, _ = img.shape
    blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), (0, 0, 0), True, crop=False)
    return img, height, width, blob

In [ ]:
# Perform object detection
def ObjectDetection(net, blob, image):
    namesOflayer = net.getnamesOflayer()
    layerOutput = [namesOflayer[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    net.setInput(blob)
    layerOutputs = net.forward(layerOutput)
    return layerOutputs

In [ ]:
# Get detection results
def GetDetectionResults(height, width, layerOutputs, classes, confThreshold=0.5, nmsThreshold=0.4):
    classIds = []
    confidences = []
    boxes = []

    for output in layerOutputs:
        for detection in output:
            score = detection[5:]
            classId = np.argmax(score)
            confidence = score[classId]

            if confidence > confThreshold:
                center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                classIds.append(classId)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)

    return boxes, confidences, classIds, indices


In [ ]:
# Draw detected objects 
def DrawDetectedObjects(img, classes, boxes, confidences, classIds, indices):
    colours = np.random.uniform(0, 255, size=(len(classes), 3))

    for i in indices:
        i = i[0]
        box = boxes[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        label = str(classes[classIds[i]])
        conf = confidences[i]
        color = colours[classIds[i]]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, f"{label} {conf:.2f}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        return img

In [ ]:
def main(imgPath, outputPath):
    net, classes = YoloLoad()
    img, h, w, blob = imgPreprocess(imgPath)
    layerOutputs = ObjectDetection(net, blob, img)
    boxes, confidences, classIds, indices = GetDetectionResults(h, w, layerOutputs, classes)
    detectedImg = DrawDetectedObjects(img, classes, boxes, confidences, classIds, indices)
    cv2.imwrite(outputPath, detectedImg)


In [ ]:
def LoadDS(dsFolderPath):
    filepathAnnot = dsFolderPath+"/annotations.csv"
    dfAnnot = pd.read_csv(filepathAnnot, sep=",")
    annotations = []
    for index, row in dfAnnot[1:].iterrows():
        annotations.append(
            dsFolderPath+"/"+row[0])

    del dfAnnot
    X_train, X_val, y_train, y_val = train_test_split(
        annotations, annotations, test_size=0.2, random_state=42)
    return  X_train, X_val, y_train, y_val

In [ ]:
X_train, X_val, y_train, y_val = LoadDS("./DataSet")
for path in X_val:
    main(path, path.split("/")[-1])